In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader



In [8]:
data_file = pd.read_csv(
    '~/ml_J1-J2_supervised/all_phase/af/augumented_dataL24.csv', index_col=[0])
print(data_file.shape)

### set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


(80000, 577)


In [62]:
### class to load the dataloader 
### this allows to iterate over the dataset
class Data_load(Dataset):
    def __init__(self, data, L, device=device):
        self.L = L
        self.x_data = torch.tensor(data.iloc[:,:int(self.L**2)].values,dtype=torch.float32).to(device=device)
        self.y_data = torch.tensor(data.iloc[:,int(self.L**2):].values,dtype=torch.long).to(device=device)
        
    def __len__(self):
        return len(self.y_data)

    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = self.x_data[idx,:]
        label = self.y_data[idx]
        return {'data':image,'lab':label}


### defining the model that we want to use
class model(nn.Module):
    def __init__(self,input_size,num_input,num_output):
        super(model,self).__init__()
        self.fc1 = nn.Linear(input_size,num_input)
        self.fc2 = nn.Linear(num_input,num_output)

    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x        

In [63]:
### set the parameters 
input_size = 576
num_input = 50
num_output = 2
learning_rate = 0.001
batch_size = 100
num_epochs = 5

In [64]:
## setting up the model
model = model(input_size,num_input,num_output).to(device=device)

## setting up the cost that one is trying to minimize
criterion = nn.CrossEntropyLoss()

## setting up the optimizer
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [65]:
### creating an object of the class dataload
train_data = Data_load(data_file, L=24, device=device)

## creating a dataloader
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)


In [67]:
for ei in range(num_epochs):
    for batch_idx,batch in enumerate(train_dataloader):
        data = batch['data'].to(device=device)
        target = batch['lab']
        target = torch.reshape(target,(-1,)).to(device=device)
        
        
        ## forward pass
        predictions = model(data)

        ## calculating the loss
        loss = criterion(predictions,target)

        ### setting all gradients to zero 
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
